In [ ]:
Looking at Psychology Today search results for 07670 - just one therapist at a time

In [1]:
from bs4 import BeautifulSoup
import requests, re, time, random
# import re
# import time
# import random

In [2]:
headers = {
    'Connection': 'keep-alive',
    'Access-Control-Request-Headers': 'content-type',
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
# Grabbing just one site - results of therapist search in ONE zip code: 07670:
myzip = ['07670']
zipurl = 'https://therapists.psychologytoday.com/rms/prof_results.php?search=' + str(myzip[0])
print(zipurl)
response_zip = requests.get(zipurl, headers=headers).text
soup_zip = BeautifulSoup(response_zip, 'html.parser')

https://therapists.psychologytoday.com/rms/prof_results.php?search=07670


In [3]:
type(soup_zip)

bs4.BeautifulSoup

In [4]:
# Manually constructing the list of urls for ALL psychotherapists in one zip - 07670:
pages_url_list = []
i = 1
while True:
    myurl = 'https://therapists.psychologytoday.com/rms/prof_results.php?sid=1507757611.7036_14284&zipcode=07670&rec_next='+str(i)
    response = requests.get(myurl, headers=headers).text
    soup = BeautifulSoup(response, 'html.parser')
    alert = soup.find('div', attrs={'class': 'alert-alert'})
    if alert is None:
        i = i + 20
        pages_url_list.append(myurl)
        # time.sleep(random.randint(1,2))
    else:
        break

In [5]:
print(pages_url_list)  # This list contains URLs of all pages that contain therapists found for this zip
print(len(pages_url_list))

['https://therapists.psychologytoday.com/rms/prof_results.php?sid=1507757611.7036_14284&zipcode=07670&rec_next=1', 'https://therapists.psychologytoday.com/rms/prof_results.php?sid=1507757611.7036_14284&zipcode=07670&rec_next=21', 'https://therapists.psychologytoday.com/rms/prof_results.php?sid=1507757611.7036_14284&zipcode=07670&rec_next=41', 'https://therapists.psychologytoday.com/rms/prof_results.php?sid=1507757611.7036_14284&zipcode=07670&rec_next=61']
4


In [6]:
# Loop through all pages of therapist search for that zip and grab the urls of each and every therapist:
therapist_urls = []
for url in pages_url_list:
    response = requests.get(url, headers = headers).text
    soup = BeautifulSoup(response, 'html.parser')
    therapists = soup.find_all('a', {'class': "result-name"})
    for i in range(len(therapists)):
        therapist_urls.append(therapists[i]["href"])
        # time.sleep(random.randint(1,2))

In [7]:
len(therapist_urls)

75

In [8]:
therapist_urls[0]

'https://therapists.psychologytoday.com/rms/prof_detail.php?profid=143075&rec_next=1&ref=1&sid=1507757611.7036_14284&zipcode=07670&tr=ResultsName'

In [51]:
# For one single therapist, grabbing info from his/her page:
response_one = requests.get(therapist_urls[32], headers=headers).text
one_therap = BeautifulSoup(response_one, 'html.parser')
# Grabbing qualifications of one therapist
qualif_div = one_therap.find('div', {'class': "profile-qualifications"})
allqualif = qualif_div.find_all('li')   # they are saved in 'li'

In [52]:
allqualif

[<li>
 <strong>Years in Practice:</strong>
             15+ Years
         </li>, <li>
 <strong>School:</strong>
             Adelphi University</li>, <li>
 <strong>Year Graduated:</strong>
             1985
         </li>, <li>
 <strong>License No. and State:</strong>
             44SC01440900 New Jersey
         </li>]

In [53]:
# Collecting Years in Practice:

years_check = [i.text.split('\n')[2].strip() for i in allqualif if "Years in Practice" in i.text]
if len(years_check) > 0:
    years_in_practice = years_check[0]
else:
    years_in_practice = ''

In [54]:
print(years_in_practice)

15+ Years


In [55]:
# Collecting School:

school_check = [i.text.split('\n')[2].strip() for i in allqualif if "School" in i.text]
if len(school_check) > 0:
    school = school_check[0]
else:
    school = ''

In [56]:
print(school)

Adelphi University


In [57]:
# Collecting Year Graduated:

yeargrad_check = [i.text.split('\n')[2].strip() for i in allqualif if "Year Graduated" in i.text]
if len(yeargrad_check) > 0:
    yeargrad = int(yeargrad_check[0])
else:
    yeargrad = ''

In [58]:
print(yeargrad)

1985


In [59]:
# Collecting Board Certification:

board_check = [i.text.split('\n')[2].strip() for i in allqualif if "Board Certification" in i.text]
if len(board_check) > 0:
    board_certif = board_check[0]
else:
    board_certif = ''

In [60]:
print(board_certif)

In [61]:
# Collecting License number & State:

license_check = [i.text.split('\n')[2].strip() for i in allqualif if "License No. and State" in i.text]
if len(license_check) > 0:
    # license = license_check[0]
    license = license_check[0].split()[0]
    license_state = " ".join(license_check[0].split()[1:])
else:
    license = ''
    license_state = ''

In [62]:
print(license)
print(license_state)

44SC01440900
New Jersey


In [63]:
# Grabbing all titles of one therapist

titles = one_therap.find('div', {'class': "profile-title"}).text

In [64]:
titles = titles.replace("\n", "")

In [65]:
titles

'Clinical Social Work/Therapist, LCSW'

In [123]:
# Grabbing financial info:
finances = one_therap.find('div', {'class': "profile-finances"})

In [125]:
# Grabbing several financial metrics:
lis = finances.find_all('li')

In [133]:
lis

[<li>
 <strong>Avg Cost (per session):</strong> $80 - $120
             </li>, <li>
 <strong>Sliding Scale:</strong> Yes
             </li>, <li>
 <strong>Accepts Insurance:</strong> Yes
             </li>, <li>
 <strong>Accepted Payment Methods: </strong>
                                     Cash,                                     Check                            </li>, <li>Aetna</li>, <li>AmeriHealth</li>, <li>BlueCross and BlueShield</li>, <li>Cigna</li>, <li>GHI</li>, <li>Health Net</li>, <li>Holman Group</li>, <li>MHN</li>, <li>Magellan Behavioral Health</li>, <li>MagnaCare</li>, <li>Medicare</li>, <li>Oxford</li>, <li>PHCS</li>, <li>TRICARE</li>, <li>UnitedHealthcare</li>, <li>ValueOptions</li>, <li>WellPoint</li>, <li>Out of Network</li>]

In [131]:
# Grab cost per session:

persession_check = [i.text.split(': ')[1].strip() for i in lis if "Avg Cost (per session)" in i.text]
if len(persession_check) > 0:
    per_session = persession_check[0]
else:
    per_session = ''

In [132]:
per_session

'$80 - $120'

In [137]:
# Grab sliding scale - yes or no:

sliding_check = [i.text.split(': ')[1].strip() for i in lis if "Sliding Scale" in i.text]
if len(sliding_check) > 0:
     sliding = sliding_check[0]
else:
     sliding = ''

In [139]:
sliding

'Yes'

In [140]:
# Grab Accepts Insurance - yes or no:

insur_check = [i.text.split(': ')[1].strip() for i in lis if "Accepts Insurance" in i.text]
if len(insur_check) > 0:
     insurance_yes = insur_check[0]
else:
     insurance_yes = ''

In [141]:
insurance_yes

'Yes'

In [155]:
# Grab Methods of Payment:

payment_check = [i.text for i in lis if "Accepted Payment Methods" in i.text]
payment_check = [i.text.split('\n')[2].strip().split(',') for i in lis if "Accepted Payment Methods" in i.text]
# if len(payment_check ) > 0:
#      payment_method = insur_check[0]
# else:
#      payment_method = ''

In [157]:
payment_check

[['Cash', '                                     Check']]